In [3]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [1]:
%pip install ultralytics
# %pip install mediapipe
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.8.17 torch-1.12.1+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
Setup complete ✅ (8 CPUs, 7.9 GB RAM, 202.2/221.5 GB disk)


In [2]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import torch

In [3]:

# mediapipe 손 감지 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils


# YOLO 객체 감지 모델 초기화
# best_model = '/content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/best.pt'
best_model = 'best.pt'
yolo_model = YOLO(best_model)


cap = cv2.VideoCapture(1) # for Mac
# cap = cv2.VideoCapture(0) # for Windows

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR 이미지를 RGB로 변환
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(frame, 1)
    results = hands.process(image)

    # YOLO 객체 감지
    box_results = yolo_model.predict(image)
    boxes = box_results[0].boxes.xyxy.cpu()
    box_class = box_results[0].boxes.cls.cpu().tolist()
    confs = box_results[0].boxes.conf.float().cpu().tolist()


    # 객체 감지 결과를 이미지에 표시
    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
              with torch.no_grad():
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)


    cv2.imshow('Frame', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 381.2ms
Speed: 11.5ms preprocess, 381.2ms inference, 9.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 eraser, 1 mouse, 267.1ms
Speed: 6.9ms preprocess, 267.1ms inference, 18.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 eraser, 1 mouse, 315.9ms
Speed: 3.6ms preprocess, 315.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 eraser, 1 mouse, 237.3ms
Speed: 4.3ms preprocess, 237.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 eraser, 1 mouse, 260.9ms
Speed: 2.5ms preprocess, 260.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 eraser, 1 mouse, 259.4ms
Speed: 0.5ms preprocess, 259.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 eraser, 281.2ms
Speed: 3.4ms preprocess, 281.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 eraser, 1 mouse, 254.4ms
Speed: 4.4m